In [41]:

#Classification

import numpy as np
import torch as t
from   matplotlib import pyplot as plt
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



data = pd.read_csv('diabetes.csv')

#print(data)

correlation_matrix = data.corr()

print(correlation_matrix)


X_col = data[['Pregnancies', 'Glucose', 'BMI', 'Age']]
Y_col = data[['Outcome']]

X = X_col.to_numpy()
Y = Y_col.to_numpy()

print("Sample x", X[:3])
print("Sample y", Y[:3])


scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

print("Sample x scaled\n", X_scaled[:3])

X_train, X_val, Y_train, Y_val = train_test_split(
    X_scaled, Y, test_size=0.2, shuffle = True, random_state=2)




                          Pregnancies   Glucose  BloodPressure  SkinThickness  \
Pregnancies                  1.000000  0.129459       0.141282      -0.081672   
Glucose                      0.129459  1.000000       0.152590       0.057328   
BloodPressure                0.141282  0.152590       1.000000       0.207371   
SkinThickness               -0.081672  0.057328       0.207371       1.000000   
Insulin                     -0.073535  0.331357       0.088933       0.436783   
BMI                          0.017683  0.221071       0.281805       0.392573   
DiabetesPedigreeFunction    -0.033523  0.137337       0.041265       0.183928   
Age                          0.544341  0.263514       0.239528      -0.113970   
Outcome                      0.221898  0.466581       0.065068       0.074752   

                           Insulin       BMI  DiabetesPedigreeFunction  \
Pregnancies              -0.073535  0.017683                 -0.033523   
Glucose                   0.331357  0.221

In [42]:
print(X_scaled.shape, X_train.shape, X_val.shape)
print(X_scaled[:3])
print(Y[:3])

(768, 4) (614, 4) (154, 4)
[[ 0.63994726  0.84832379  0.20401277  1.4259954 ]
 [-0.84488505 -1.12339636 -0.68442195 -0.19067191]
 [ 1.23388019  1.94372388 -1.10325546 -0.10558415]]
[[1]
 [0]
 [1]]


In [43]:
# convert data from numpy to tensors
X_t_train = t.from_numpy(X_train).float()
Y_t_train = t.flatten(t.from_numpy(Y_train).long()) # flatten - creates a one dimensional tensor
X_t_val   = t.from_numpy(X_val).float()
Y_t_val   = t.flatten(t.from_numpy(Y_val).long())

print(X_t_train.shape, Y_t_train.shape)
print(X_t_train.dtype, Y_t_train.dtype)
print(X_t_val.shape,   Y_t_val.shape)
print(X_t_val.dtype,   Y_t_val.dtype)

torch.Size([614, 4]) torch.Size([614])
torch.float32 torch.int64
torch.Size([154, 4]) torch.Size([154])
torch.float32 torch.int64


In [44]:
# DataLoader = a class that shuffles the data and splits in into batches
# you should use it during training (SGD - accumulate error over batches of data )
train_data = [(X_t_train[i], Y_t_train[i]) for i in range(X_t_train.shape[0])]
print("Sample train_data = ", train_data[:3], " type = ", type(train_data))
trainloader = t.utils.data.DataLoader(train_data, batch_size = 10, shuffle=True)
for x,label in trainloader:  # shuffles the data
    print(x,label)

Sample train_data =  [(tensor([-0.8449,  0.0972, -0.5321, -0.2758]), tensor(0)), (tensor([-0.8449, -0.8104,  0.3817,  0.8304]), tensor(1)), (tensor([-1.1419, -0.8417,  1.4605, -1.0415]), tensor(0))]  type =  <class 'list'>
tensor([[ 1.5308e+00,  1.5369e+00,  1.5240e+00,  8.3038e-01],
        [ 1.8278e+00, -9.0432e-01, -7.7327e-01, -1.9067e-01],
        [ 9.3691e-01,  2.0689e+00,  2.4209e-01,  6.4591e-02],
        [ 1.2339e+00, -2.7996e-02, -8.8749e-01,  2.6172e+00],
        [-8.4489e-01, -6.8524e-01,  8.3861e-01, -1.0415e+00],
        [-5.4792e-01,  6.5895e-02,  1.2828e+00, -6.1611e-01],
        [ 1.8278e+00,  1.2849e-01, -1.1329e-01,  6.6021e-01],
        [ 9.3691e-01,  9.7351e-01,  2.2855e+00,  2.3477e-01],
        [-1.1419e+00, -9.0591e-02, -4.0605e+00, -1.0415e+00],
        [ 9.3691e-01,  5.0406e-01,  9.4198e-04,  4.9003e-01]]) tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
tensor([[ 1.5308,  1.0987, -0.9129,  1.6813],
        [ 1.2339,  1.8811, -0.2402,  2.2769],
        [-0.2510,  0.2850

In [45]:
class Model(nn.Module):
    def __init__(self, input_dim = 4):
         
        super(Model, self).__init__()
        self.layer1   = nn.Linear(in_features=input_dim, out_features = 15)
        self.dropout1 = nn.Dropout(p = 0.3) # drop 30% of output nodes from the previous layer during training only 
        self.layer2   = nn.Linear(in_features= 15, out_features = 12)
        self.dropout2 = nn.Dropout(p = 0.25)
        self.layer3   = nn.Linear(in_features = 12, out_features = 9) # 3 neurons = one for each class
        self.dropout3 = nn.Dropout(p=0.2)
        self.layer4   = nn.Linear(in_features= 9, out_features= 3)

        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        x = F.relu(self.layer3(x))
        x = self.dropout3(x)
        x = self.layer4(x)
        
        return x

In [46]:
model     = Model(X_train.shape[1])   # X_train.shape[1]
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
print(model)

Y0 = model.forward(X_t_train[0,:])
print(Y0)

Model(
  (layer1): Linear(in_features=4, out_features=15, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (layer2): Linear(in_features=15, out_features=12, bias=True)
  (dropout2): Dropout(p=0.25, inplace=False)
  (layer3): Linear(in_features=12, out_features=9, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (layer4): Linear(in_features=9, out_features=3, bias=True)
)
tensor([-0.2795,  0.0292,  0.2404], grad_fn=<ViewBackward0>)


In [47]:
def train_model(n_epochs, model, train_loader, optimizer, loss_fn, 
                X_val, Y_val):
    for epoch in range(n_epochs+1):
        for xb,yb in train_loader: # for each batch
            model.train()       # set model in training mode = with dropout
            ym = model.forward(xb)
            loss = loss_fn(ym,yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            model.eval() # set model in evaluation mode = no dropout
            with t.no_grad():  # no learning
                ym_val   = model.forward(X_val)
                loss_val = loss_fn(ym_val,Y_val)

            print(f"Epoch {epoch}, Training loss {loss.item():.4f},"
f" Validation loss {loss_val.item():.4f}")

In [48]:
train_model(n_epochs = 1000, model = model, 
            train_loader = trainloader, optimizer = optimizer, 
            loss_fn = loss_fn, 
            X_val   = X_t_val,   Y_val = Y_t_val)

Epoch 0, Training loss 0.8942, Validation loss 1.0828
Epoch 1, Training loss 0.5958, Validation loss 0.7376
Epoch 100, Training loss 0.1873, Validation loss 0.4709
Epoch 200, Training loss 0.6013, Validation loss 0.4755
Epoch 300, Training loss 0.9076, Validation loss 0.4739
Epoch 400, Training loss 0.2402, Validation loss 0.4818
Epoch 500, Training loss 0.2765, Validation loss 0.4879
Epoch 600, Training loss 0.7160, Validation loss 0.4922
Epoch 700, Training loss 0.1405, Validation loss 0.4909
Epoch 800, Training loss 0.4303, Validation loss 0.4969
Epoch 900, Training loss 0.4243, Validation loss 0.4909
Epoch 1000, Training loss 0.1151, Validation loss 0.4930


In [50]:

soft_max  = t.nn.Softmax(1) 
y_m_train = soft_max(model.forward(X_t_train)) # y_m normalized with softmax
y_m_train = t.argmax(y_m_train,dim = 1)

y_m_val   = soft_max(model.forward(X_t_val))
y_m_val   = t.argmax(y_m_val,dim = 1)
correct_pred_val = t.sum(y_m_val == Y_t_val)/y_m_val.shape[0]
print("Validation accuracy = ", correct_pred_val)

correct_pred_train = t.sum(y_m_train == Y_t_train)/y_m_train.shape[0]
print("Training accuracy = ", correct_pred_train)

Validation accuracy =  tensor(0.7922)
Training accuracy =  tensor(0.8013)
